# The demonstration of the tropical precipitation module on the different datasets

###  Loading the necessary packages

In [1]:
from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import importlib

import inspect
import timeit
import sys

### Function, which reads the status of the user in the slurm queue 

In [2]:
def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

### Storing the path to the current repository into the variable

In [3]:
with os.popen("pwd ") as f:
    _pwd = f.readline()

pwd = re.split(r'[\n]', _pwd)[0]

In [21]:
pwd

'/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/time_memory_estimation'

### Setting the slurm job

In [70]:

extra_args=[
    "--error="+str(pwd)+"/slurm/logs/dask-worker-%j.err",
    "--output="+str(pwd)+"/slurm/output/dask-worker-%j.out"
]

cluster = SLURMCluster(
    name='dask-cluster', 
    cores=1,    
    memory="100 GB", 
    project="bb1153",
    queue= "interactive",#"computi", 
    walltime='01:30:50',
    #job_extra=extra_args,
)
client = Client(cluster)
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p interactive
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=94G
#SBATCH -t 01:30:50

/home/b/b382267/mambaforge/envs/tropical-rainfall_2/bin/python -m distributed.cli.dask_worker tcp://136.172.124.2:37449 --nthreads 1 --memory-limit 93.13GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/tropical-rainfall_2/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall_2/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40339 instead
  warnings.warn(


### Loading the slurm job to the queue 

In [71]:
cluster.scale(jobs=1)

### Checking our slurm job in the queue 

In [73]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4640869 interacti dask-wor  b382267  R       0:19      1 l40056
           4638647 interacti interact  b382267  R    2:13:01      1 l40053


0

In [14]:
import runpy

In [23]:
_res = %timeit -r 2 -n 1 -o runpy.run_path(path_name='test.py')  #runpy.run_module(mod_name='test') #
result = _res
time_fast_reg, time_fast_reg_stdev =  result.average, result.stdev 

B 5
B 5
5.01 s ± 578 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [25]:
import runpy

In [30]:
_t = timeit.timeit("run_path(path_name='test.py')", setup="from runpy import run_path",  number=2)
_t = _t/2
_t

B 5
B 5


5.008053312078118

In [11]:
from test import SIZE

In [12]:
SIZE

5

### Canceling the job if it is necessary. Set the job ID manually

In [8]:
Job_ID =   4523604
os.system("scancel " +str(Job_ID)) 

0

### Setting the path to the diagnostic repository 

In [21]:
sys.path.append(str(pwd)+'/../../')
import src.shared_func
import src.tr_pr_mod
import src.time_estimation
import src.time_estimation_opt
#import src.time_memory_estimator

### Loading the extra function, which can be used in any diagnostic

In [22]:
while True:
    try:
        importlib.reload(src.shared_func)
        from  src.shared_func import time_interpreter,  animation_creator, image_creator,  xarray_attribute_update, data_size
        break
    except NameError and AttributeError:
        import src.shared_func
        from  src.shared_func import time_interpreter,   animation_creator, image_creator,  xarray_attribute_update, data_size
        break

### Loading the tropical precipitation module

In [23]:
while True:
    try:
        importlib.reload(src.tr_pr_mod)
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break
    except NameError and AttributeError:
        import src.tr_pr_mod
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break

In [24]:
while True:
    try:
        importlib.reload(src.time_estimation)
        from  src.time_estimation import  estimated_total_calc_time,   optimal_amout_of_timesteps, time_units_converter
        break
    except NameError and AttributeError:
        import src.time_estimation
        from  src.time_estimation import  estimated_total_calc_time,   optimal_amout_of_timesteps,  time_units_converter
        break

In [68]:
while True:
    try:
        importlib.reload(src.time_estimation_opt)
        from  src.time_estimation_opt import  estimated_total_calc_time_opt,   optimal_amout_of_timesteps_opt
        break
    except NameError and AttributeError:
        import src.time_estimation_opt
        from  src.time_estimation_opt import  estimated_total_calc_time_opt,   optimal_amout_of_timesteps_opt

In [27]:
configdir = '../../../../config/'
diagname  = 'tr_pr'
machine   = 'levante'

### Setting the class attributes

In [28]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-4)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

### Importing the aqua module 

In [29]:
import aqua

In [30]:
from aqua import Reader
from aqua.reader import catalogue

In [31]:
catalogue(configdir=configdir)

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- lra-r100-day	
	- lra-r100-mon	
IFS	test-tco2559	4km experiment, coupled with FESOM
	- ICMGG_atm2d	2d output
	- ICMU_atm3d	3d output

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng

levante:
  args:
    path: ../../../../config//levante/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


# ICON data

In [32]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4638719 interacti dask-wor  b382267  R       0:55      1 l40056
           4638647 interacti interact  b382267  R       9:30      1 l40053


0

In [33]:
reader = Reader(model="ICON", exp="ngc2009",  configdir=configdir, source="atm_2d_ml_R02B09", regrid="r200")
ICON_2009_reg = reader.retrieve()

In [46]:
ICON_2009_reg 

<xarray.Dataset>
Dimensions:   (time: 37009, cell: 20971520)
Coordinates:
  * time      (time) datetime64[ns] 2020-01-20 ... 2022-03-01
Dimensions without coordinates: cell
Data variables: (12/33)
    cfh_lnd   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tciw      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tclw      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    cptgzvi   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mslhfl    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msshfl    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    ...        ...
    10u       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    10v       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msntrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mtntrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msnsrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mtnsrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-04-06 10:35:42 retrie...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

## Time check

In [ ]:
ICON_chunk = ICON_2009_reg['tprate'][0:20,:]

##  Checking the calculation time of a small chunk of the dataset

In [108]:
%timeit -r 2 -n 1 -o  diag.hist1d_fast(ICON_chunk,  preprocess = False)

3.28 s ± 30 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 3.28 s ± 30 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

### Average calculation time and error

In [109]:
result = _
time_fast_reg, time_fast_reg_stdev =  result.average, result.stdev 

In [110]:
time_fast_reg

3.27808556612581

### Estemated calculation time of entire dataset

In [116]:
tot_calc_time = estimated_total_calc_time(ds_part = ICON_chunk, calc_time = time_fast_reg, ds_full = ICON_2009_reg['tprate'])

### Converting obtained calculation time into comfortable units

In [179]:
print(tot_calc_time, 's')
time_units_converter(old_value = tot_calc_time, desirable_time_unit = 'H')

6065.933435837505 s


1.6849815099548626

### Checking how many timesteps we need to load to finish the diagnostic by a particular time

In [181]:
optimal_amout_of_timesteps(ds_part = ICON_chunk, calc_time = time_fast_reg, wanted_time = '1H', ds_full = ICON_2009_reg['tprate'])

(0.5934783225168969, 21964)

In [ ]:
time_interpreter(ICON_chunk)

## Optimized

In [55]:
estimated_total_calc_time_opt( ICON_2009_reg['tprate'])

32196.325509619448

In [74]:
optimal_amout_of_timesteps_opt(wanted_time='2H', ds_full = ICON_2009_reg['tprate'])

/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/time_memory_estimation


(0.1949546787430091, 7215)